In [1]:
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
import pandas as pd
import statsmodels.stats.api as sms
import numpy as np, scipy.stats as st
from statsmodels.stats.proportion import samplesize_confint_proportion

In [2]:
df = pd.read_csv('water.txt', sep="\t", header=0)#, names=["location", "town", "mortality","hardness"])
df["mortality"] = pd.to_numeric(df["mortality"], downcast='float')
df["hardness"] = pd.to_numeric(df["hardness"], downcast='float')
#data.columns = ["location", "town", "mortality","hardness"]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   location   61 non-null     object 
 1   town       61 non-null     object 
 2   mortality  61 non-null     float32
 3   hardness   61 non-null     float32
dtypes: float32(2), object(2)
memory usage: 1.6+ KB


In [4]:
df.head()

,location,town,mortality,hardness
0,South,Bath,1247.0,105.0
1,North,Birkenhead,1668.0,17.0
2,South,Birmingham,1466.0,5.0
3,North,Blackburn,1800.0,14.0
4,North,Blackpool,1609.0,18.0


## Question 2

Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его **нижняя граница**? Округлите ответ до 4 знаков после десятичной точки.

[link to stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data)

In [5]:
mortality = pd.DataFrame.to_numpy(df["mortality"])
mortality

array([1247., 1668., 1466., 1800., 1609., 1558., 1807., 1299., 1637.,
       1359., 1392., 1755., 1519., 1307., 1254., 1491., 1555., 1428.,
       1318., 1260., 1723., 1379., 1742., 1574., 1569., 1096., 1591.,
       1402., 1772., 1828., 1704., 1702., 1581., 1309., 1259., 1427.,
       1724., 1175., 1486., 1456., 1696., 1236., 1711., 1444., 1591.,
       1987., 1495., 1369., 1257., 1587., 1713., 1557., 1640., 1709.,
       1625., 1625., 1527., 1627., 1486., 1485., 1378.], dtype=float32)

In [6]:
df["mortality"].std(ddof=1)

187.66876220703125

z-интервал - **не столило считать, так как нет дисперсии**

In [7]:
sms.DescrStatsW(mortality).tconfint_mean()

(1476.0833412806621, 1572.2117406865511)

In [8]:
st.norm.interval(0.95, loc=np.mean(mortality), scale=st.sem(mortality))

(1477.0525475985103, 1571.2426184171147)

t-интервал - у нас нет дисперсии, используем ее приближение

In [9]:
mort_mean_std = df["mortality"].std() / np.sqrt(df["mortality"].shape[0])
print('Mortality south 95%% interval: %s' %  str(_tconfint_generic(df["mortality"].mean(), mort_mean_std,
                                                                   df["mortality"].shape[0] - 1,
                                                                   0.05, 'two-sided')))

Mortality south 95% interval: (1476.0833812600179, 1572.2117847556071)


In [10]:
_tconfint_generic(df['mortality'].mean(), 
                  df['mortality'].std(ddof=1) / np.sqrt(len(df)),  len(df) - 1, 0.05, 'two-sided') 

(1476.0833812600179, 1572.2117847556071)

## Question 3

На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем **южным городам**. Чему равна его **верхняя граница**? Округлите ответ до 4 знаков после десятичной точки.

In [11]:
mort_south = pd.DataFrame.to_numpy(df.loc[df["location"] ==  "South"]["mortality"])
mort_north = pd.DataFrame.to_numpy(df.loc[df["location"] ==  "North"]["mortality"])

#df.loc[df["location"] ==  "South"]["mortality"].std(ddof=1)
df_south = df.loc[df["location"] ==  "South"]["mortality"]
df_north = df.loc[df["location"] ==  "North"]["mortality"]

In [12]:
st.norm.interval(0.95, loc=np.mean(mort_south), scale=st.sem(mort_south))

(1322.8909929516128, 1430.7244855640122)

t-интвервал - у нас нет дисперсии

In [13]:
mort_mean_south_std = df_south.std() / np.sqrt(df_south.shape[0])
print('Mortality south 95%% interval: %s' %  str(_tconfint_generic(df_south.mean(), mort_mean_south_std,
                                                                   df_south.shape[0] - 1,
                                                                   0.05, 'two-sided')))

Mortality south 95% interval: (1320.1517912238, 1433.463687291825)


In [14]:
_tconfint_generic(df[df.location == 'South'].mortality.mean(),     df[df.location == 'South'].mortality.std(ddof=1) / np.sqrt(len(df[df.location == 'South'])), len(df[df.location == 'South']) - 1, 0.05, 'two-sided')

(1320.1517912238, 1433.463687291825)

## Question 4

На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод? 

In [15]:
st.norm.interval(0.95, loc=np.mean(mort_north), scale=st.sem(mort_north))

(1588.2335485578133, 1678.9664026140617)

In [16]:
df.loc[df["location"] ==  "North"]["mortality"].std(ddof=1)

136.93690490722656

## Question 5

Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?

In [17]:
_tconfint_generic(df[df.location == 'South'].hardness.mean(), df[df.location == 'South'].hardness.std(ddof=1) / np.sqrt(len(df[df.location == 'South'])), len(df[df.location == 'South']) - 1, 0.05, 'two-sided')
_tconfint_generic(df[df.location == 'North'].hardness.mean(), df[df.location == 'North'].hardness.std(ddof=1) / np.sqrt(len(df[df.location == 'North'])), len(df[df.location == 'North']) - 1, 0.05, 'two-sided')

(21.422487867791236, 39.377511369269314)

## Question 6
Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией $σ2\sigma^2σ2:$

$Xˉn±z1−α2σn\bar{X}_n\pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}Xˉn​±z1−2α​​n$

​σ​

При $σ=1\sigma=1σ=1$ какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью ±0.1\pm0.1±0.1?

In [18]:
#n_samples = int(np.ceil(samplesize_confint_proportion(random_sample.mean(), 0.01)))
#n_samples

NameError: name 'random_sample' is not defined

In [19]:
from scipy import stats
np.ceil((stats.norm.ppf(1-0.05/2) / 0.1)**2)

385.0